In [ ]:
# default_exp exporters.exporters
%load_ext autoreload
%autoreload 2

In [ ]:
# export
# hide
from typing import Dict, List, Optional, Iterable, Any
import pandas as pd
import json

from pymemri.pod.client import PodClient
from pymemri.data.itembase import Item

# Query

With the `Query` class you can query the Pod with a list of properties, and export the result to a tabular dataformat. These properties can include edges that point to different items.

In [ ]:
# export
class Query:
    def __init__(self, *properties: List[str]):
        """
        A Query implements functionality to retrieve data from the pod to a tabular format.
        
        Given a list of `properties`, the `execute` method queries the pod for a set of given items, 
        and retrieves the properties for each item if it exists. Note that a properties can be nested behind
        multiple edges, such as "sender.owner.firstName".
        """
        self.properties = list(properties)

    def traverse_edges(self, client: PodClient, items: List[Item], edges: List[str]) -> List[Item]:
        items = items.copy()

        for edge in edges:
            ids_to_query = list()
            query_item_idx = list()
            for i in range(len(items)):
                item = items[i]
                if item is None:
                    continue
                # Replace item with target item. If the edge is empty, it has to be queried again.
                try:
                    if edge not in item.edges:
                        items[i] = None
                    else:
                        items[i] = getattr(item, edge)[0]
                except Exception:
                    ids_to_query.append(item.id)
                    query_item_idx.append(i)
                    items[i] = None

            
            new_items = client.search({"ids": ids_to_query})
            for i, new_item in zip(query_item_idx, new_items):
                try:
                    items[i] = getattr(new_item, edge)[0]
                except Exception:
                    items[i] = None
        return items

    def get_property_values(
        self, client: PodClient, prop: str, items: List[Item]
    ) -> list:
        edges, prop_name = self.parse_property(prop)
        target_items = self.traverse_edges(client, items, edges)

        result = [getattr(item, prop_name, None) for item in target_items]
        return result

    @staticmethod
    def parse_property(prop: str):
        prop = prop.split(".")
        edges = prop[:-1]
        prop = prop[-1]
        return edges, prop

    def convert_dtype(self, result, dtype):
        if dtype == "dict":
            return result
        elif dtype == "list":
            return [result[prop] for prop in self.properties]
        elif dtype in {"pandas", "pd", "df"}:
            return pd.DataFrame.from_dict(result)
        else:
            raise ValueError(f"Unknown dtype: {dtype}")

    def execute(self, client: PodClient, items: List[Item], dtype="dict") -> Any:
        result = {
            prop: self.get_property_values(client, prop, items) for prop in self.properties
        }
        return self.convert_dtype(result, dtype)

## Tests

In [ ]:
# hide
from pymemri.pod.client import PodClient
from pymemri.data.schema import Account, Person, Message, Label
from pymemri.data.itembase import Edge
import random

### Create dummy data for dataset-

In [ ]:
# hide
client = PodClient()
client.add_to_schema(Account, Person, Message)

num_items = 10
messages = []
items = []
edges = []
for i in range(num_items):   
    msg = Message(content=f"content_{i}", service="my_service")
    account = Account(handle=f"account_{i}")
    person = Person(firstName=f"firstname_{i}")
    label = Label(name=f"label_{i}")
    items.extend([msg, account, person, label])
    edges.extend([
        Edge(msg, account, "sender"),
        Edge(msg, label, "label"),
        Edge(account, person, "owner")
    ])
    messages.append(msg)
    
# Dataset is not perfect, drop some random edges
edges = random.sample(edges, int(len(edges)*0.8))

client.bulk_action(
    create_items=items,
    create_edges=edges
)

BULK: Writing 64/64 items/edges
Completed Bulk action, written 64 items/edges


True

### Example usage
For example, if we have a list of Messages and we want to get message content,
the accompanying account handles and account owner names, we can query:

In [ ]:
messages = client.search({"type": "Message", "service": "my_service"})
query = Query("content", "sender.handle", "sender.owner.firstName")
query.execute(client, messages, dtype="pandas")

,content,sender.handle,sender.owner.firstName
0,content_4,account_4,firstname_4
1,content_1,account_1,None
2,content_2,account_2,firstname_2
3,content_3,account_3,firstname_3
4,content_5,account_5,firstname_5
5,content_6,account_6,firstname_6
6,content_0,account_0,firstname_0
7,content_7,account_7,None
8,content_8,account_8,firstname_8
9,content_9,account_9,None


In [ ]:
q = Query("content", "label.name", "sender.owner.firstName", "sender.handle", "wrong_property")
result = q.execute(client, messages)
result

{'content': ['content_4',
  'content_1',
  'content_2',
  'content_3',
  'content_5',
  'content_6',
  'content_0',
  'content_7',
  'content_8',
  'content_9'],
 'label.name': [None,
  'label_1',
  'label_2',
  None,
  'label_5',
  'label_6',
  'label_0',
  'label_7',
  None,
  'label_9'],
 'sender.owner.firstName': ['firstname_4',
  None,
  'firstname_2',
  'firstname_3',
  'firstname_5',
  'firstname_6',
  'firstname_0',
  None,
  'firstname_8',
  None],
 'sender.handle': ['account_4',
  'account_1',
  'account_2',
  'account_3',
  'account_5',
  'account_6',
  'account_0',
  'account_7',
  'account_8',
  'account_9'],
 'wrong_property': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None]}

In [ ]:
# hide
q = Query("content", "label.name", "sender.owner.firstName", "sender.handle", "wrong_property")
result = q.execute(client, messages)

assert all(len(vals) == len(result["content"]) for vals in result.values())
assert len(result["content"]) == num_items

# Check if columns all align
valid_props = ["label.name", "sender.owner.firstName", "sender.handle"]
for i in range(num_items):
    row = [result[prop][i] for prop in valid_props]
    row_idx = [val[-1] for val in row if val is not None]
    assert len(set(row_idx)) <= 1

assert all(val is None for val in result["wrong_property"])

In [ ]:
q = Query("content", "label.name", "sender.owner.firstName", "sender.handle")
result = q.execute(client, messages, dtype="pandas")
result.head()

,content,label.name,sender.owner.firstName,sender.handle
0,content_0,label_0,None,None
1,content_1,label_1,firstname_1,account_1
2,content_2,None,firstname_2,account_2
3,content_3,label_3,None,account_3
4,content_4,label_4,None,None


# Exporting Datasets



In [ ]:
# export
class Dataset(Item):
    """
    Temporary dataset schema, remove when MVP2 is done.
    """
    properties= Item.properties + ["queryStr"]
    edges = Item.edges + ["item"]
    
    def __init__(self, queryStr: str = None, item: list = None, **kwargs):
        super().__init__(**kwargs)
        self.queryStr = queryStr
        self.item: list = item if item is not None else []

In [ ]:
# export
def filter_missing(dataset: dict) -> dict:
    missing_idx = set()
    for column in dataset.values():
        missing_idx.update([i for i, val in enumerate(column) if val is None])
    return {
        k: [item for i, item in enumerate(v) if i not in missing_idx] for k, v in dataset.items()
    }

def export_dataset(
    client: PodClient,
    dataset: Dataset,
    content_fields: List[str] = ["content"],
    label_field: str = "label.value",
    missing_values: bool = False,
    dtype: str = "dict",
):
    items = dataset.item
    query = Query("id", *content_fields, label_field)
    result = query.execute(client, dataset.item)
    
    if not missing_values:
        result = filter_missing(result)
        
    return query.convert_dtype(result, dtype)

In [ ]:
# hide
client.add_to_schema(Dataset)
search_query = json.dumps({"type": "Message"})

messages = [client.get(msg.id) for msg in messages]
dataset = Dataset(search_query)
edges = [
    Edge(dataset, msg, "item") for msg in messages
]

client.bulk_action(
    create_items = [dataset],
    create_edges = edges
)

BULK: Writing 11/11 items/edges
Completed Bulk action, written 11 items/edges


True

In [ ]:
# hide
dataset = client.get(dataset.id)

df = export_dataset(client, dataset, dtype="pandas")
df

,id,content,label.name
0,96edc0082df740b8a704ceb5c7447f13,content_0,label_0
1,55f8381205ef4ac98e47e17254aa26e1,content_1,label_1
2,4adcf318e6254947b113b8d65d387dc2,content_3,label_3
3,10b8af9fcf9e4a619bba7310083dcdd2,content_4,label_4
4,ea76f206a6b1422aaef77162abceb1a7,content_5,label_5
5,2cd1dedf4fa245feaec64c3ceddd3ba6,content_6,label_6
6,62ce7bda27fc4347b284e1d1329ad785,content_7,label_7
7,eeeecc2165ed4051aaa248aa03458e0d,content_8,label_8
8,4f7389b314cf46559dac6601f8d46588,content_9,label_9


In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted Untitled.ipynb.
Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted exporters.exporters.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.listeners.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.api.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted template.config.ipynb.
Converted template.formatter.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
